### Offline Setup & Locate Dataset (Recursive)

In [ ]:
import os, numpy as np, matplotlib.pyplot as plt
import mne
from scipy.signal import butter, filtfilt, iirnotch, welch
from scipy.stats import skew, kurtosis

INPUT_ROOT = "/kaggle/input"
print("Mounted inputs:", [d for d in os.listdir(INPUT_ROOT) if os.path.isdir(os.path.join(INPUT_ROOT, d))])

ROOT_CAND = os.path.join(INPUT_ROOT, "physiobank-database-sleep-edfx-cassette")
assert os.path.isdir(ROOT_CAND), "Không thấy thư mục 'physiobank-database-sleep-edfx-cassette' sau khi Add Input."

# Tìm tất cả thư mục con có .edf
edf_dirs = {}
for root, dirs, files in os.walk(ROOT_CAND):
    edfs = [f for f in files if f.lower().endswith(".edf")]
    if edfs:
        edf_dirs[root] = edfs

if not edf_dirs:
    raise FileNotFoundError("Không tìm thấy file .edf trong dataset đã gắn. Kiểm tra lại dataset đã Add.")

# Chọn thư mục có nhiều .edf nhất làm DATA_PATH
DATA_PATH = max(edf_dirs.keys(), key=lambda k: len(edf_dirs[k]))
files = sorted(edf_dirs[DATA_PATH])

print("DATA_PATH =", DATA_PATH)
print("Total EDF files in this folder:", len(files))
print("Sample files:", files[:12])

### Select PSG/Hypnogram Subset for Processing

In [ ]:
SUBSET_PAIRS = [
    ("SC4001E0-PSG.edf", "SC4001EC-Hypnogram.edf"),
    ("SC4002E0-PSG.edf", "SC4002EC-Hypnogram.edf"),
    ("SC4011E0-PSG.edf", "SC4011EH-Hypnogram.edf"),
]
for psg, hyp in SUBSET_PAIRS:
    assert psg in files and hyp in files, f"Thiếu cặp: {psg} / {hyp}"
print("Subset OK:", SUBSET_PAIRS)

### Load One PSG/Hyp Pair and Inspect Channels & Labels

In [ ]:
psg_file = os.path.join(DATA_PATH, SUBSET_PAIRS[0][0])
hyp_file = os.path.join(DATA_PATH, SUBSET_PAIRS[0][1])

raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)
ann = mne.read_annotations(hyp_file)
print("Channels:", raw.ch_names)
print("Sampling rate:", raw.info['sfreq'])
print("Unique annotations:", sorted({a['description'] for a in ann})[:10])

### Select Main EEG Channel and Visualize Raw Signal

In [ ]:
raw.pick(['EEG Fpz-Cz'])  
fs = raw.info['sfreq']

x_10s = raw.get_data()[0][:int(fs*10)]
plt.figure(figsize=(10,3)); plt.plot(x_10s)
plt.title("Raw EEG (Fpz-Cz) – first 10s"); plt.xlabel("Sample"); plt.ylabel("a.u.")
plt.show()

### Digital Filtering (Band-pass 0.5–40 Hz + Notch 50 Hz)

In [ ]:
def bandpass_filter(x, fs, lo=0.5, hi=40, order=4):
    from scipy.signal import butter, filtfilt
    b,a = butter(order, [lo/(fs/2), hi/(fs/2)], btype='band'); return filtfilt(b, a, x)

def notch_filter(x, fs, f0=50, Q=30):
    from scipy.signal import iirnotch, filtfilt
    b,a = iirnotch(f0/(fs/2), Q); return filtfilt(b, a, x)

x = raw.get_data()[0]
x_filt = notch_filter(bandpass_filter(x, fs, 0.5, 40, 4), fs, f0=50, Q=30)

plt.figure(figsize=(10,3)); plt.plot(x_filt[:int(fs*10)])
plt.title("Filtered EEG (first 10s)"); plt.xlabel("Sample"); plt.ylabel("a.u.")
plt.show()

### Frequency Spectrum Analysis (FFT of EEG Signal)

In [ ]:
from scipy.fft import fft, fftfreq

N = int(10*fs)
x_seg = x_filt[:N]
Y = fft(x_seg*np.hanning(N))
F = fftfreq(N, 1/fs)

plt.figure(figsize=(7,3))
plt.plot(F[:N//2], 2.0/N*np.abs(Y[:N//2]))
plt.xlim(0, 60)
plt.xlabel("Frequency (Hz)"); plt.ylabel("Amplitude")
plt.title("EEG Fpz–Cz Spectrum (10 s segment)")
plt.grid(True); plt.show()

### Discrete Filter Analysis: Butterworth Band-pass (0.5–40 Hz)

In [ ]:
from scipy.signal import butter, freqz

b, a = butter(4, [0.5/(fs/2), 40/(fs/2)], btype='band')
w, h = freqz(b, a, worN=2048)

plt.figure(figsize=(6,3))
plt.plot((w/np.pi)*(fs/2), 20*np.log10(np.maximum(np.abs(h),1e-12)))
plt.title("Butterworth Band-pass 0.5–40 Hz (order 4)")
plt.xlabel("Frequency (Hz)"); plt.ylabel("Magnitude (dB)"); plt.grid(True); plt.show()

p = np.roots(a); z = np.roots(b)
theta = np.linspace(0, 2*np.pi, 512)
plt.figure(figsize=(3.5,3.5))
plt.plot(np.cos(theta), np.sin(theta), 'k--')
plt.scatter(z.real, z.imag, marker='o', label='Zeros')
plt.scatter(p.real, p.imag, marker='x', label='Poles')
plt.axis('equal'); plt.title("Pole–Zero Diagram"); plt.legend(); plt.show()

### Sampling Rate Reduction (Decimation & Anti-Aliasing Check)

In [ ]:
from scipy.signal import firwin, lfilter
target_fs = 80
lp_cut = 35.0
numtaps = 255
lp = firwin(numtaps, lp_cut/(fs/2))
x_aa = lfilter(lp, [1.0], x_filt)
factor = int(round(fs/target_fs))
x_dec = x_aa[::factor]
fs_dec = fs/factor
print("Decimated sampling rate:", fs_dec)

### EOG Artifact Removal using ICA (Multi-channel)

In [ ]:
import mne
from mne.preprocessing import ICA, create_eog_epochs

# 1) đọc lại PSG dạng multi-channel (đừng dùng 'raw' đã pick 1 kênh)
raw_multi = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)
fs_mc = raw_multi.info['sfreq']

# 2) chọn kênh EEG+EOG nếu có
picks = mne.pick_types(raw_multi.info, eeg=True, eog=True, exclude=[])
n_ch = len(picks)
print(f"[ICA] channels available: {n_ch}")

def _bp_notch(x, fs):
    # dùng cùng tham số lọc như Cell 5
    from scipy.signal import butter, filtfilt, iirnotch
    b,a = butter(4, [0.5/(fs/2), 40/(fs/2)], btype='band')
    x = filtfilt(b,a,x)
    b,a = iirnotch(50/(fs/2), 30)
    x = filtfilt(b,a,x)
    return x

if n_ch < 2:
    print("[ICA] <2 channels → skip ICA, use x_filt.")
    x_sig = x_filt.copy()
else:
    # 3) high-pass 1 Hz trước khi fit ICA (khuyến nghị MNE)
    raw_hp = raw_multi.copy().filter(l_freq=1.0, h_freq=None, picks=picks, method="iir", verbose=False)

    # 4) số thành phần ICA không vượt quá số kênh
    n_comp = min(15, n_ch)
    ica = ICA(n_components=n_comp, random_state=42, method='fastica')
    ica.fit(raw_hp, picks=picks, verbose=False)
    print(f"[ICA] fitted with n_components={n_comp}")

    # 5) tìm thành phần liên quan EOG
    has_eog = len(mne.pick_types(raw_multi.info, eog=True)) > 0
    if has_eog:
        eog_chs = [ch for ch in raw_hp.ch_names if 'EOG' in ch.upper()]
        eog_epochs = create_eog_epochs(raw_hp, ch_name=(eog_chs[0] if eog_chs else None),
                                       reject_by_annotation=True, verbose=False)
        eog_inds, _ = ica.find_bads_eog(eog_epochs, threshold=2.0)
    else:
        # fallback: tương quan với EEG chính
        eeg_main = 'EEG Fpz-Cz' if 'EEG Fpz-Cz' in raw_hp.ch_names else raw_hp.ch_names[mne.pick_types(raw_hp.info, eeg=True, exclude=[])[0]]
        eog_inds, _ = ica.find_bads_eog(raw_hp, ch_name=eeg_main, threshold=2.0)

    ica.exclude = eog_inds
    print("[ICA] excluded comps:", eog_inds)

    # 6) áp dụng ICA và rút kênh EEG chính
    raw_clean_mc = ica.apply(raw_hp.copy(), verbose=False)
    eeg_name = 'EEG Fpz-Cz' if 'EEG Fpz-Cz' in raw_clean_mc.ch_names else raw_clean_mc.ch_names[mne.pick_types(raw_clean_mc.info, eeg=True, exclude=[])[0]]
    raw_clean_eeg = raw_clean_mc.copy().pick([eeg_name])
    x_clean0 = raw_clean_eeg.get_data()[0]

    # 7) band-pass + notch (để nhất quán với pipeline) → x_sig
    x_sig = _bp_notch(x_clean0, fs_mc)

print("x_sig ready. Downstream steps (epoch/features) will use x_sig.")

### Epoching (30 s Windows) and Label Mapping

In [ ]:
EPOCH_SEC = 30
label_map = {
    'Sleep stage W':0, 'Sleep stage 1':1, 'Sleep stage 2':2,
    'Sleep stage 3':3, 'Sleep stage 4':3, 'Sleep stage R':4
}

# đặt x_sig vào raw tạm của kênh EEG chính
raw_tmp = raw.copy()                
raw_tmp._data[0] = x_sig           
raw_tmp.set_annotations(ann)

events, _ = mne.events_from_annotations(raw_tmp, event_id=label_map, verbose=False)
epochs = mne.Epochs(raw_tmp, events, tmin=0, tmax=EPOCH_SEC, baseline=None, preload=True, verbose=False)

y_subject = epochs.events[:, 2]
import pandas as pd
print(epochs)
print("Label counts:", pd.Series(y_subject).value_counts().sort_index().to_dict())

### Time–Frequency Analysis (STFT & Multitaper PSD)

In [ ]:
x_ep = epochs.get_data()[0,0,:]
plt.figure(figsize=(6,3))
Pxx, freqs, bins, im = plt.specgram(x_ep, NFFT=256, Fs=fs, noverlap=128)
plt.ylim(0,40)
plt.xlabel("Time (s)")
plt.ylabel("Hz")
plt.title("Spectrogram - Epoch 0")
plt.colorbar(label='Power')
plt.show()

NFFT = 256
nover = int(0.75*NFFT)
plt.figure(figsize=(6,3))
Pxx, freqs, bins, im = plt.specgram(x_ep, NFFT=NFFT, Fs=fs, noverlap=nover)
plt.ylim(0,40); plt.xlabel("Time (s)"); plt.ylabel("Hz")
plt.title("STFT (NFFT = 256, Overlap = 75%)")
plt.colorbar(label="Power"); plt.show()

from mne.time_frequency import psd_array_multitaper
psd_mt, f_mt = psd_array_multitaper(x_ep, sfreq=fs, fmin=0.5, fmax=40.0,
                                    adaptive=True, normalization='full', verbose=False)
plt.figure(figsize=(6,3))
plt.semilogy(f_mt, psd_mt)
plt.xlabel("Hz"); plt.ylabel("PSD"); plt.title("Multitaper PSD – Sample Epoch")
plt.grid(True); plt.show()

### Advanced Feature Extractor (Time + Spectral + Rich DSP)

In [ ]:
from scipy.signal import welch
from scipy.stats import skew, kurtosis
import numpy as np

BANDS = {"delta":(0.5,4), "theta":(4,8), "alpha":(8,12), "beta":(12,30), "gamma":(30,40)}

def rich_spectral_features(x, fs):
    f, Pxx = welch(x, fs=fs, nperseg=int(4*fs))
    m = (f>=0.5) & (f<=40); f=f[m]; P=Pxx[m]; P = P + 1e-18
    total = np.trapz(P, f)
    centroid  = np.trapz(f*P, f)/total
    bandwidth = np.sqrt(np.trapz(((f-centroid)**2)*P, f)/total)
    csum = np.cumsum(P)
    median_f = f[np.searchsorted(csum, 0.5*csum[-1])]
    sef90    = f[np.searchsorted(csum, 0.9*csum[-1])]
    # 1/f correction (log-log detrend)
    logf, logP = np.log(f), np.log(P)
    A = np.vstack([logf, np.ones_like(logf)]).T
    slope, intercept = np.linalg.lstsq(A, logP, rcond=None)[0]
    P_corr = np.exp(logP - (slope*logf + intercept))
    hf = np.trapz(P_corr[(f>=12)&(f<=30)], f[(f>=12)&(f<=30)])
    lf = np.trapz(P_corr[(f>=0.5)&(f<=4)], f[(f>=0.5)&(f<=4)])
    hf_lf_ratio = hf/(lf+1e-12)
    return {"spectral_centroid":centroid, "spectral_bandwidth":bandwidth,
            "median_freq":median_f, "sef90":sef90, "hf_lf_1overf":hf_lf_ratio}

def extract_features_epoch_ADV(x, fs):
    # z-score per epoch
    x = (x - np.mean(x)) / (np.std(x) + 1e-8)
    feats = {
        "mean": float(np.mean(x)),
        "std":  float(np.std(x)),
        "var":  float(np.var(x)),
        "skew": float(skew(x)),
        "kurt": float(kurtosis(x, fisher=True)),
    }
    # Hjorth
    dx, ddx = np.diff(x), np.diff(np.diff(x))
    vx = np.var(x) + 1e-12; vdx = np.var(dx) + 1e-12; vddx = np.var(ddx) + 1e-12
    feats.update({
        "hjorth_activity": vx,
        "hjorth_mobility": np.sqrt(vdx/vx),
        "hjorth_complexity": (np.sqrt(vddx/vdx)) / (np.sqrt(vdx/vx) + 1e-12),
    })
    # Welch band powers
    f, Pxx = welch(x, fs=fs, nperseg=int(4*fs))
    mask = (f>=0.5)&(f<=40); total = np.trapz(Pxx[mask], f[mask]) + 1e-12
    for name,(lo,hi) in BANDS.items():
        idx = (f>=lo)&(f<=hi)
        p = np.trapz(Pxx[idx], f[idx])
        feats[f"pow_{name}_abs"] = p
        feats[f"pow_{name}_rel"] = p/total
    # spectral entropy (custom, normalized)
    P = Pxx[mask]; Pn = P/np.sum(P); H = -np.sum(Pn*np.log(Pn+1e-12))/np.log(len(Pn))
    feats["spec_entropy"] = float(H)
    # rich spectral
    feats.update(rich_spectral_features(x, fs))
    return feats

### Extract Features for All Epochs (One Subject)

In [ ]:
# === Cell 14: features for one subject using ADV extractor ===
X_subj, y_subj = [], []
for i in range(len(epochs)):
    x_ep = epochs.get_data()[i, 0, :]   # 1 channel
    feats = extract_features_epoch_ADV(x_ep, fs)
    if i == 0:
        feature_names = list(feats.keys())
    X_subj.append([feats[k] for k in feature_names])
    y_subj.append(int(epochs.events[i, 2]))

X_subj = np.asarray(X_subj, dtype=np.float32)
y_subj = np.asarray(y_subj, dtype=np.int64)

print("X_subj:", X_subj.shape, "| y_subj:", y_subj.shape, "| n_features:", len(feature_names))
print("All finite? ", np.isfinite(X_subj).all())

### Batch Processing & Save (Base vs ICA Datasets)

In [ ]:
import json, pandas as pd, numpy as np
import mne

def process_one_pair(psg_name, hyp_name, use_ica=False):
    psg_path = os.path.join(DATA_PATH, psg_name)
    hyp_path = os.path.join(DATA_PATH, hyp_name)

    # ---- read multi-channel of THIS subject (do NOT reuse global 'raw') ----
    raw_mc = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
    fs_local = raw_mc.info['sfreq']

    # choose main EEG name for this file
    if 'EEG Fpz-Cz' in raw_mc.ch_names:
        eeg_name = 'EEG Fpz-Cz'
    else:
        eeg_idx = mne.pick_types(raw_mc.info, eeg=True, exclude=[])[0]
        eeg_name = raw_mc.ch_names[eeg_idx]

    # --- helper: our pipeline band-pass + notch (same as earlier) ---
    from scipy.signal import butter, filtfilt, iirnotch
    def bp_notch(x, fs):
        b,a = butter(4, [0.5/(fs/2), 40/(fs/2)], btype='band'); x = filtfilt(b,a,x)
        b,a = iirnotch(50/(fs/2), 30);                               x = filtfilt(b,a,x)
        return x

    # --- get EEG series (optionally with ICA) *from this subject* ---
    def get_eeg_series(use_ica_flag):
        if not use_ica_flag:
            sig0 = raw_mc.copy().pick([eeg_name]).get_data()[0]
            return bp_notch(sig0, fs_local)

        picks = mne.pick_types(raw_mc.info, eeg=True, eog=True, exclude=[])
        if len(picks) < 2:
            # not enough channels for ICA, fall back to base
            sig0 = raw_mc.copy().pick([eeg_name]).get_data()[0]
            return bp_notch(sig0, fs_local)

        # high-pass 1 Hz then ICA
        raw_hp = raw_mc.copy().filter(l_freq=1.0, h_freq=None, picks=picks, method="iir", verbose=False)
        n_comp = min(15, len(picks))
        ica = mne.preprocessing.ICA(n_components=n_comp, random_state=42, method='fastica')
        ica.fit(raw_hp, picks=picks, verbose=False)

        has_eog = len(mne.pick_types(raw_mc.info, eog=True)) > 0
        if has_eog:
            eog_chs = [ch for ch in raw_hp.ch_names if 'EOG' in ch.upper()]
            eog_epochs = mne.preprocessing.create_eog_epochs(raw_hp, ch_name=(eog_chs[0] if eog_chs else None),
                                                             reject_by_annotation=True, verbose=False)
            eog_inds, _ = ica.find_bads_eog(eog_epochs, threshold=2.0)
        else:
            eog_inds, _ = ica.find_bads_eog(raw_hp, ch_name=eeg_name, threshold=2.0)
        ica.exclude = eog_inds

        raw_clean_mc = ica.apply(raw_hp.copy(), verbose=False)
        sig0 = raw_clean_mc.copy().pick([eeg_name]).get_data()[0]
        return bp_notch(sig0, fs_local)

    x_series = get_eeg_series(use_ica)

    # ---- build raw_tmp FROM raw_mc (same subject, same length) ----
    raw_eeg = raw_mc.copy().pick([eeg_name])      # single-channel Raw
    assert raw_eeg.get_data().shape[1] == x_series.shape[0], \
        f"Length mismatch: raw={raw_eeg.get_data().shape[1]} vs x_series={x_series.shape[0]}"

    raw_tmp = raw_eeg.copy()
    raw_tmp._data[0] = x_series  # now lengths match

    # annotations of THIS subject
    ann = mne.read_annotations(os.path.join(DATA_PATH, hyp_name))
    raw_tmp.set_annotations(ann)

    # epoch 30 s with 5-class mapping
    label_map = {'Sleep stage W':0,'Sleep stage 1':1,'Sleep stage 2':2,'Sleep stage 3':3,'Sleep stage 4':3,'Sleep stage R':4}
    events, _ = mne.events_from_annotations(raw_tmp, event_id=label_map, verbose=False)
    epochs_local = mne.Epochs(raw_tmp, events, tmin=0, tmax=30, baseline=None, preload=True, verbose=False)

    # feature extraction (ADV)
    X_list, y_list = [], []
    for i in range(len(epochs_local)):
        x_ep = epochs_local.get_data()[i,0,:]
        feats = extract_features_epoch_ADV(x_ep, fs_local)
        if i == 0: fn = list(feats.keys())
        X_list.append([feats[k] for k in fn])
        y_list.append(int(epochs_local.events[i,2]))

    return np.array(X_list, np.float32), np.array(y_list, np.int64), fn

def run_batch(use_ica=False, tag="base"):
    X_all, y_all, feature_names, meta = None, None, None, []
    for psg, hyp in SUBSET_PAIRS:
        sid = psg.split('-')[0]
        Xs, ys, fn = process_one_pair(psg, hyp, use_ica=use_ica)
        if feature_names is None: feature_names = fn
        X_all = Xs if X_all is None else np.vstack([X_all, Xs])
        y_all = ys if y_all is None else np.concatenate([y_all, ys])
        meta += [(sid, i, int(ys[i])) for i in range(len(ys))]

    np.savez(f"/kaggle/working/features_{tag}.npz", X=X_all, y=y_all)
    with open(f"/kaggle/working/feature_names_{tag}.json","w") as f: json.dump(feature_names, f)
    pd.DataFrame(meta, columns=["subject_id","epoch_idx","stage"]).to_csv(f"/kaggle/working/meta_{tag}.csv", index=False)
    print(f"Saved: features_{tag}.npz, feature_names_{tag}.json, meta_{tag}.csv")
    print("Shapes:", X_all.shape, y_all.shape)
    return X_all, y_all

# --- Run both variants for ablation ---
Xb, yb = run_batch(use_ica=False, tag="base")
Xi, yi = run_batch(use_ica=True,  tag="ica")

### CONFIG & IMPORT

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import time
import json

from tqdm.auto import tqdm

from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_validate
)
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, f1_score, cohen_kappa_score, classification_report,
    confusion_matrix, make_scorer
)

# --- Experiment Configurations ---
RANDOM_STATE = 42
TEST_SIZE = 0.2
USE_GROUP_SPLIT = False       # Set to True if meta has a 'subject' column
CV_FOLDS = 5
SCORING_MAIN = "f1_macro"     # main metric to rank models
CLASS_WEIGHT = None           # or "balanced" if you prefer F1/Kappa

DATA_PATH = ""                # Folder containing features_base.npz, meta_base.csv, ...

# Scoring dict for cross-validation
kappa_scorer = make_scorer(cohen_kappa_score)
SCORING_DICT = {
    "accuracy": "accuracy",
    "f1_macro": "f1_macro",
    "cohen_kappa": kappa_scorer,
}

# Create required folders
os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

print("Configuration loaded.")

### READ DATA

In [ ]:
def load_data(name):
    """Load X, labels, and feature names for each dataset."""
    # Load features
    with np.load(os.path.join(DATA_PATH, f"features_{name}.npz")) as f:
        X = f["X"]

    # Load metadata
    df_meta = pd.read_csv(os.path.join(DATA_PATH, f"meta_{name}.csv"))
    y_raw = df_meta["stage"].values

    # Load feature names
    with open(os.path.join(DATA_PATH, f"feature_names_{name}.json"), "r") as f:
        feature_names = json.load(f)

    print(f"{name.upper()} loaded: X={X.shape}, y={y_raw.shape}")
    return X, y_raw, feature_names


X_base, y_base_raw, feature_names_base = load_data("base")
X_ica,  y_ica_raw,  feature_names_ica  = load_data("ica")

### LABEL ENCODING

In [ ]:
# Fit on union of Base ∪ ICA to ensure consistent mapping
label_union = np.unique(np.concatenate([y_base_raw, y_ica_raw]))
le = LabelEncoder()
le.fit(label_union)

y_base = le.transform(y_base_raw)
y_ica  = le.transform(y_ica_raw)

print("Label mapping (string → ID):")
print(dict(zip(le.classes_, le.transform(le.classes_))))

### TRAIN/TEST SPLIT

In [ ]:
indices = np.arange(len(X_base))

train_idx, test_idx = train_test_split(
    indices,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y_base
)

X_train_base, X_test_base = X_base[train_idx], X_base[test_idx]
y_train_base, y_test_base = y_base[train_idx], y_base[test_idx]

X_train_ica, X_test_ica = X_ica[train_idx], X_ica[test_idx]
y_train_ica, y_test_ica = y_ica[train_idx], y_ica[test_idx]

print(f"Split completed: Train={len(train_idx)}, Test={len(test_idx)}")

### DEFINE MODELS & PIPELINES

In [ ]:
# Base models (no tuning yet)
MODELS = {
    "Logistic Regression": LogisticRegression(
        random_state=RANDOM_STATE,
        max_iter=2000,
        class_weight=CLASS_WEIGHT,
        solver="lbfgs",
        multi_class="auto"
    ),
    "Random Forest": RandomForestClassifier(
        random_state=RANDOM_STATE,
        n_estimators=300,
        class_weight=CLASS_WEIGHT
    ),
    "SVM (RBF)": SVC(
        kernel="rbf",
        class_weight=CLASS_WEIGHT
    )
}

# Pipelines for baseline comparison (CV on BASE)
pipelines = {
    name: Pipeline([
        ("scaler", StandardScaler()),
        ("clf", model)
    ])
    for name, model in MODELS.items()
}

# Hyperparameter grid for Logistic Regression (main model)
LR_PARAM_GRID = {
    "C": [0.001, 0.01, 0.1, 1, 10],
    "solver": ["liblinear", "lbfgs"]
}

print("Models, pipelines, and LR hyperparameter grid are ready.")

### CROSS-VALIDATION (BASE, ALL MODELS)

In [ ]:
cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)

cv_results = []

print(f"Running {CV_FOLDS}-fold cross-validation on BASE features for all models...")

for name, pipe in pipelines.items():
    print(f"\n>>> Model: {name}")
    start_time_cv = time.time()

    scores = cross_validate(
        pipe,
        X_train_base,
        y_train_base,
        cv=cv,
        scoring=SCORING_DICT,
        n_jobs=-1
    )

    time_taken = time.time() - start_time_cv

    acc_mean = scores["test_accuracy"].mean()
    f1_mean = scores["test_f1_macro"].mean()
    kappa_mean = scores["test_cohen_kappa"].mean()

    cv_results.append({
        "Model": name,
        "accuracy_mean": acc_mean,
        "f1_macro_mean": f1_mean,
        "kappa_mean": kappa_mean,
        "time_sec": time_taken
    })

    print(f"Accuracy (mean): {acc_mean:.4f}")
    print(f"F1-macro (mean): {f1_mean:.4f}")
    print(f"Cohen's kappa:  {kappa_mean:.4f}")
    print(f"CV time:         {time_taken:.2f} s")

df_cv = pd.DataFrame(cv_results)
print("\n=== CV SUMMARY (BASE FEATURES) ===")
print(df_cv)

# Optionally, find the best model by F1_macro (for information)
best_by_f1 = df_cv.loc[df_cv["f1_macro_mean"].idxmax()]
print("\nBest model on CV (by F1_macro):")
print(best_by_f1)

### LR GRID SEARCH (WITH PROGRESS BAR)

In [ ]:
from sklearn.model_selection import ParameterGrid

param_list = list(ParameterGrid(LR_PARAM_GRID))
best_score = -np.inf
best_params = None

print(f"Tuning Logistic Regression ({len(param_list)} combinations)...")

start_time_lr_tuning = time.time()    # <-- START TIMER

for params in tqdm(param_list, desc="LR Grid Search", unit="combination"):
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            random_state=RANDOM_STATE,
            max_iter=2000,
            class_weight=CLASS_WEIGHT,
            **params
        ))
    ])

    fold_scores = []
    for tr_idx, val_idx in cv.split(X_train_base, y_train_base):
        X_tr, X_val = X_train_base[tr_idx], X_train_base[val_idx]
        y_tr, y_val = y_train_base[tr_idx], y_train_base[val_idx]

        pipe.fit(X_tr, y_tr)
        y_pred = pipe.predict(X_val)

        f1 = f1_score(y_val, y_pred, average="macro", zero_division=0)
        fold_scores.append(f1)

    mean_f1 = np.mean(fold_scores)

    if mean_f1 > best_score:
        best_score = mean_f1
        best_params = params

# <-- END TIMER
tuning_time = time.time() - start_time_lr_tuning

print("\nBest LR params:", best_params)
print("Best CV F1_macro:", best_score)
print(f"Tuning time: {tuning_time:.2f} seconds")

### FINAL TRAIN (ALL MODELS, BASE & ICA)

In [ ]:
results_base = {}
results_ica = {}

# 1) Build final LR models (with best_params from Cell 6)
best_lr_base = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        random_state=RANDOM_STATE,
        max_iter=2000,
        class_weight=CLASS_WEIGHT,
        **best_params
    ))
])

best_lr_ica = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        random_state=RANDOM_STATE,
        max_iter=2000,
        class_weight=CLASS_WEIGHT,
        **best_params
    ))
])

# 2) Build RF & SVM pipelines (using same configs as in MODELS)
rf_base = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(
        random_state=RANDOM_STATE,
        n_estimators=300,
        class_weight=CLASS_WEIGHT
    ))
])

rf_ica = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(
        random_state=RANDOM_STATE,
        n_estimators=300,
        class_weight=CLASS_WEIGHT
    ))
])

svm_base = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SVC(
        kernel="rbf",
        class_weight=CLASS_WEIGHT
    ))
])

svm_ica = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SVC(
        kernel="rbf",
        class_weight=CLASS_WEIGHT
    ))
])

# 3) Fit all models on BASE
start_train_base = time.time()
best_lr_base.fit(X_train_base, y_train_base)
rf_base.fit(X_train_base, y_train_base)
svm_base.fit(X_train_base, y_train_base)
train_time_base = time.time() - start_train_base

# 4) Fit all models on ICA
start_train_ica = time.time()
best_lr_ica.fit(X_train_ica, y_train_ica)
rf_ica.fit(X_train_ica, y_train_ica)
svm_ica.fit(X_train_ica, y_train_ica)
train_time_ica = time.time() - start_train_ica

# 5) Evaluate helper
def eval_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "f1_macro": f1_score(y_test, y_pred, average="macro", zero_division=0),
        "f1_weighted": f1_score(y_test, y_pred, average="weighted", zero_division=0),
        "kappa": cohen_kappa_score(y_test, y_pred),
        "y_pred": y_pred,
    }

# 6) Evaluate all on BASE
results_base["Logistic Regression"] = eval_model(best_lr_base, X_test_base, y_test_base)
results_base["Random Forest"]      = eval_model(rf_base,     X_test_base, y_test_base)
results_base["SVM (RBF)"]          = eval_model(svm_base,    X_test_base, y_test_base)

# 7) Evaluate all on ICA
results_ica["Logistic Regression"] = eval_model(best_lr_ica, X_test_ica, y_test_ica)
results_ica["Random Forest"]       = eval_model(rf_ica,      X_test_ica, y_test_ica)
results_ica["SVM (RBF)"]           = eval_model(svm_ica,     X_test_ica, y_test_ica)

# 8) Print summary
print("\n=== FINAL TEST RESULTS (BASE) ===")
for name, res in results_base.items():
    print(f"\nModel: {name}")
    print(f"Accuracy:      {res['accuracy']:.4f}")
    print(f"F1_macro:      {res['f1_macro']:.4f}")
    print(f"F1_weighted:   {res['f1_weighted']:.4f}")
    print(f"Cohen's Kappa: {res['kappa']:.4f}")

print("\n=== FINAL TEST RESULTS (ICA) ===")
for name, res in results_ica.items():
    print(f"\nModel: {name}")
    print(f"Accuracy:      {res['accuracy']:.4f}")
    print(f"F1_macro:      {res['f1_macro']:.4f}")
    print(f"F1_weighted:   {res['f1_weighted']:.4f}")
    print(f"Cohen's Kappa: {res['kappa']:.4f}")

# 9) Determine best model by F1_macro
best_model_base_name = max(results_base.items(), key=lambda x: x[1]["f1_macro"])[0]
best_model_ica_name  = max(results_ica.items(),  key=lambda x: x[1]["f1_macro"])[0]

print(f"\nBest model on BASE (by F1_macro): {best_model_base_name}")
print(f"Best model on ICA  (by F1_macro): {best_model_ica_name}")

### VISUALIZATION

In [ ]:
def plot_cm(y_true, y_pred, title, path):
    cm = confusion_matrix(y_true, y_pred, normalize="true")
    plt.figure(figsize=(7,6))
    sns.heatmap(cm, annot=True, cmap="Blues", fmt=".2f",
                xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(path)
    plt.show()

# Bar chart: F1_macro Base vs ICA
model_names = list(results_base.keys())
f1_base = [results_base[m]["f1_macro"] for m in model_names]
f1_ica  = [results_ica[m]["f1_macro"]  for m in model_names]

x = np.arange(len(model_names))
width = 0.35

plt.figure(figsize=(8,6))
plt.bar(x - width/2, f1_base, width, label="BASE")
plt.bar(x + width/2, f1_ica,  width, label="ICA")
plt.xticks(x, model_names, rotation=15)
plt.ylabel("F1-macro")
plt.title("Model Comparison: BASE vs ICA (F1-macro)")
plt.legend()
plt.tight_layout()
plt.savefig("results/f1macro_base_vs_ica.png")
plt.show()

# Confusion matrix for best model (BASE)
best_base_pred = results_base[best_model_base_name]["y_pred"]
plot_cm(
    y_test_base,
    best_base_pred,
    f"BASE | Best model: {best_model_base_name}",
    "results/cm_base_best.png"
)

# Confusion matrix for best model (ICA)
best_ica_pred = results_ica[best_model_ica_name]["y_pred"]
plot_cm(
    y_test_ica,
    best_ica_pred,
    f"ICA | Best model: {best_model_ica_name}",
    "results/cm_ica_best.png"
)

In [ ]:
training_log = {
    "lr_grid_search_time_sec": tuning_time,             
    "lr_final_train_base_sec": train_time_base,         
    "lr_final_train_ica_sec": train_time_ica,           
}

training_log["total_pipeline_time_sec"] = (
    training_log["lr_grid_search_time_sec"]
    + training_log["lr_final_train_base_sec"]
    + training_log["lr_final_train_ica_sec"]
)

for k, v in training_log.items():
    print(f"{k:<35}: {v:.2f} s")

# save to json
json.dump(training_log, open("results/training_time_log.json", "w"), indent=4)

print("\nTraining time log saved to results/training_time_log.json")

### SAVE RESULTS

In [ ]:
json.dump(best_params, open("results/best_params.json", "w"), indent=4)
print("Saved best_params.json")

### SAVE MODELS

In [ ]:
def safe_name(name: str) -> str:
    """Convert model name into a filesystem-safe string."""
    return (
        name.lower()
        .replace(" ", "_")
        .replace("(", "")
        .replace(")", "")
        .replace("/", "_")
    )

# Map model names to actual trained model objects (BASE)
models_base = {
    "Logistic Regression": best_lr_base,
    "Random Forest": rf_base,
    "SVM (RBF)": svm_base,
}

# Map model names to actual trained model objects (ICA)
models_ica = {
    "Logistic Regression": best_lr_ica,
    "Random Forest": rf_ica,
    "SVM (RBF)": svm_ica,
}

# Save all models for BASE
joblib.dump(best_lr_base, "models/logistic_regression_base.pkl")
joblib.dump(rf_base,       "models/random_forest_base.pkl")
joblib.dump(svm_base,      "models/svm_rbf_base.pkl")

# Save all models for ICA
joblib.dump(best_lr_ica, "models/logistic_regression_ica.pkl")
joblib.dump(rf_ica,      "models/random_forest_ica.pkl")
joblib.dump(svm_ica,     "models/svm_rbf_ica.pkl")

print("Saved all models for BASE and ICA.")

# Also mark and save the best models explicitly (by F1_macro)
best_base_filename = f"models/best_model_base_{safe_name(best_model_base_name)}.pkl"
best_ica_filename  = f"models/best_model_ica_{safe_name(best_model_ica_name)}.pkl"

joblib.dump(models_base[best_model_base_name], best_base_filename)
joblib.dump(models_ica[best_model_ica_name],   best_ica_filename)

print(f"Best BASE model ({best_model_base_name}) saved to: {best_base_filename}")
print(f"Best ICA  model ({best_model_ica_name}) saved to: {best_ica_filename}")

# Optionally save a small JSON summary of which model is best
best_model_summary = {
    "base": {
        "best_model_name": best_model_base_name,
        "file": best_base_filename,
    },
    "ica": {
        "best_model_name": best_model_ica_name,
        "file": best_ica_filename,
    },
}

with open("results/best_models_summary.json", "w") as f:
    json.dump(best_model_summary, f, indent=4)

print("Best model summary saved to results/best_models_summary.json")

### VERIFY MODEL EXPORT

In [ ]:
# Load best model summary
with open("results/best_models_summary.json", "r") as f:
    best_info = json.load(f)

best_base_file = best_info["base"]["file"]
best_ica_file  = best_info["ica"]["file"]

print(f"Best BASE model file: {best_base_file}")
print(f"Best ICA  model file: {best_ica_file}")

# Load best BASE model
best_base_model = joblib.load(best_base_file)
sample_base = X_test_base[0].reshape(1, -1)
pred_base_id = best_base_model.predict(sample_base)[0]
pred_base_label = le.inverse_transform([pred_base_id])[0]

print("\n[BASE] Single-sample prediction:")
print(f"Predicted ID:    {pred_base_id}")
print(f"Predicted label: {pred_base_label}")

# Load best ICA model
best_ica_model = joblib.load(best_ica_file)
sample_ica = X_test_ica[0].reshape(1, -1)
pred_ica_id = best_ica_model.predict(sample_ica)[0]
pred_ica_label = le.inverse_transform([pred_ica_id])[0]

print("\n[ICA] Single-sample prediction:")
print(f"Predicted ID:    {pred_ica_id}")
print(f"Predicted label: {pred_ica_label}")

print("\nModel export verification completed successfully.")

### EXPORT LABEL MAP

In [ ]:
import json
import os

print("=== EXPORT LABEL MAP ===")

# Original mapping (string label -> numeric ID)
label_map = {
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3': 3,
    'Sleep stage 4': 3,   # merged into N3 (AASM standard)
    'Sleep stage R': 4
}

# Build inverse mapping (numeric ID -> label string)
inverse_label_map = {}
for label_str, label_id in label_map.items():
    # Keep only the first occurrence for each numeric ID
    if label_id not in inverse_label_map:
        inverse_label_map[label_id] = label_str

# Convert integer keys to string keys for JSON compatibility
inverse_label_map_str_keys = {str(k): v for k, v in inverse_label_map.items()}

print("--- Inverse Label Map (ID → Label) ---")
print(inverse_label_map_str_keys)

# Save to results/label_map.json
output_path = "results/label_map.json"

try:
    with open(output_path, "w") as f:
        json.dump(inverse_label_map_str_keys, f, indent=4)
    print(f"\n Label mapping successfully saved at: {output_path}")
except FileNotFoundError:
    print("\n WARNING: 'results/' directory not found. Please create it before saving.")